In [6]:
import pandas as pd


# Adjusted function to extract and save specific columns based on the exact column names
def save_specific_columns_v3(excel_data, sheet_names):
    csv_files = []
    for sheet in sheet_names:
        df = pd.read_excel(excel_data, sheet_name=sheet)

        # Extract and save specific columns for each person
        for person in ['Patrick', 'Jiacheng', 'Zifeng']:
            # Construct the column name for the person
            person_column = f"{sheet.strip()} ({person})"
            if person_column in df.columns:
                # Find the index of the person's column
                start_index = df.columns.get_loc(person_column)
                # Select the person's column and the next four columns
                selected_columns = df.iloc[:, start_index:start_index + 5]

                # Save to CSV
                csv_filename = f'./temp/{sheet.strip()}_for_{person}.csv'
                selected_columns.to_csv(csv_filename, index=False)
                csv_files.append(csv_filename)

    return csv_files




In [7]:
# Load the Excel file
file_path = '/home/pj20/GREScore/human_eval/GRE_human_eval.xlsx'
excel_data = pd.ExcelFile(file_path)

# Display the names of the sheets in the Excel file
sheet_names = excel_data.sheet_names

# Convert the sheets to CSVs with specific columns
csv_specific_file_paths_v3 = save_specific_columns_v3(excel_data, sheet_names)
csv_specific_file_paths_v3

['./temp/Topical_for_Patrick.csv',
 './temp/Topical_for_Jiacheng.csv',
 './temp/Topical_for_Zifeng.csv',
 './temp/Uniqueness_for_Patrick.csv',
 './temp/Uniqueness_for_Jiacheng.csv',
 './temp/Uniqueness_for_Zifeng.csv',
 './temp/Factualness_for_Patrick.csv',
 './temp/Factualness_for_Jiacheng.csv',
 './temp/Factualness_for_Zifeng.csv',
 './temp/Granularity_for_Patrick.csv',
 './temp/Granularity_for_Jiacheng.csv',
 './temp/Granularity_for_Zifeng.csv',
 './temp/Completeness_for_Patrick.csv',
 './temp/Completeness_for_Jiacheng.csv',
 './temp/Completeness_for_Zifeng.csv']

In [8]:
import os
# Function to create pairwise comparisons between models
def create_pairwise_comparisons(df):
    # List of models for comparison
    model_columns = df.columns[1:]  # Excluding the sample ID column

    # Create an empty DataFrame for the pairwise comparisons
    comparisons_df = pd.DataFrame(columns=["sample_id", "model_A_name", "model_B_name", "win"])

    # Iterate through each row in the DataFrame
    for _, row in df.iterrows():
        # Get the sample ID
        sample_id = row.iloc[0]

        # Compare each model with every other model
        for i in range(len(model_columns)):
            for j in range(i+1, len(model_columns)):
                model_a = model_columns[i]
                model_b = model_columns[j]
                rank_a = row[model_a]
                rank_b = row[model_b]

                # Determine the winner
                if rank_a < rank_b:
                    win = 'model_A_win'
                elif rank_a > rank_b:
                    win = 'model_B_win'
                else:
                    win = 'tie'

                # Add the comparison to the DataFrame
                comparison = {"sample_id": sample_id, "model_A_name": model_a, "model_B_name": model_b, "win": win}
                comparisons_df = pd.concat([comparisons_df, pd.DataFrame([comparison])], ignore_index=True)


    return comparisons_df


In [9]:
# laod files in csv_specific_file_paths_v3
for csv_file in csv_specific_file_paths_v3:
    df = pd.read_csv(csv_file)
    
    # Create the pairwise comparison DataFrame
    pairwise_comparisons_df = create_pairwise_comparisons(df)

    # change the column name if the column_name end with .1 or .2, remove it
    pairwise_comparisons_df['model_A_name'] = pairwise_comparisons_df['model_A_name'].str.replace('.1', '')
    pairwise_comparisons_df['model_B_name'] = pairwise_comparisons_df['model_B_name'].str.replace('.1', '')
    pairwise_comparisons_df['model_A_name'] = pairwise_comparisons_df['model_A_name'].str.replace('.2', '')
    pairwise_comparisons_df['model_B_name'] = pairwise_comparisons_df['model_B_name'].str.replace('.2', '')

    # obtain the filename
    csv_filename = os.path.basename(csv_file)
    csv_filename = csv_filename.replace('.csv', '_pairwise_comparisons.csv')

    print(csv_filename)
    pairwise_comparisons_df.to_csv(os.path.join('results', csv_filename), index=False)

Topical_for_Patrick_pairwise_comparisons.csv
Topical_for_Jiacheng_pairwise_comparisons.csv
Topical_for_Zifeng_pairwise_comparisons.csv
Uniqueness_for_Patrick_pairwise_comparisons.csv
Uniqueness_for_Jiacheng_pairwise_comparisons.csv
Uniqueness_for_Zifeng_pairwise_comparisons.csv
Factualness_for_Patrick_pairwise_comparisons.csv
Factualness_for_Jiacheng_pairwise_comparisons.csv
Factualness_for_Zifeng_pairwise_comparisons.csv
Granularity_for_Patrick_pairwise_comparisons.csv
Granularity_for_Jiacheng_pairwise_comparisons.csv
Granularity_for_Zifeng_pairwise_comparisons.csv
Completeness_for_Patrick_pairwise_comparisons.csv
Completeness_for_Jiacheng_pairwise_comparisons.csv
Completeness_for_Zifeng_pairwise_comparisons.csv
